In [1]:
from __future__ import division
import tensorflow as tf
from hourglass_tiny import HourglassModel
import scipy.io
from my_utils import *
from tensorflow.python.ops import gen_nn_ops
import os
import matplotlib.pyplot as plt
import pandas as pd

C:\Users\44163\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


In [2]:
@tf.RegisterGradient("GuidedRelu")
def _GuidedReluGrad(op, grad):
    return tf.where(0. < grad, gen_nn_ops._relu_grad(grad, op.outputs[0]), tf.zeros_like(grad))
g = tf.get_default_graph()
with g.gradient_override_map({"Relu": "GuidedRelu"}):
    print('--Parsing Config File')
    params = process_config('config.cfg')
    model = HourglassModel(nFeat=params['nfeats'], nStack=params['nstacks'],
                       nModules=params['nmodules'],
                       nLow=params['nlow'], outputDim=params['num_joints'],
                       batch_size=params['batch_size'], attention = params['mcam'],
                       training=True, drop_rate= params['dropout_rate'],
                       lear_rate=params['learning_rate'],
                       decay=params['learning_rate_decay'],
                       decay_step=params['decay_step'],
                       name=params['name'],
                       logdir_train=params['log_dir_train'],
                       logdir_test=params['log_dir_test'],
                       tiny= params['tiny'], w_loss=params['weighted_loss'] ,
                       joints= params['joint_list'],modif=False)
    model.generate_model()
    # restore old net to continue training
    path = "trained_nets/net_with_surreal_mar_26"
    full_path = os.path.join(path,'save_net.ckpt')

    # output: (None,4,64,64,15)
    output = model.get_output()
    # get input
    inpt = model.img
    # get output
    out = output[0,3]
    pred_seg = tf.argmax(out,axis=2)
   
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    saver = tf.train.Saver()
    saver.restore(sess,full_path) 

--Parsing Config File
CREATE MODEL:
---Inputs : Done (0 sec.)


---Graph : Done (12 sec.)
---Loss : Done (0 sec.)


INFO:tensorflow:Restoring parameters from trained_nets/net_with_surreal_mar_26\save_net.ckpt


# 1.load data

## surreal

In [7]:
surreal_data = pd.read_csv('surreal_data_table.csv')
surreal_data.head()
i = 3003
img = image.img_to_array(image.load_img(surreal_data['jpg'][i]))
label = scipy.io.loadmat(surreal_data['mat'][i])['mask']
cut1,cut2,cut3 = random_put_into_256map(img,label)
test_img = cut3[:,:,0:3]
plt.figure()
plt.imshow(test_img/255)
plt.show()
input_img = np.expand_dims(test_img,axis=0)

## pascal

In [ ]:
pascal_data = pd.read_csv('pacal_data_inform.csv')
pascal_data.head()
i = 3427
img = image.img_to_array(image.load_img(pascal_data['jpg_path'][i]))
label = scipy.io.loadmat(pascal_data['mat_path'][i])['single_map']
cut1,cut2,cut3 = random_put_into_256map(img,label)
test_img = cut3[:,:,0:3]
plt.figure()
plt.imshow(test_img/255)
plt.show()
input_img = np.expand_dims(test_img,axis=0)

## Sitting

In [56]:
sitting_data = pd.read_csv('sitting_data_table.csv')
sitting_data.head()
i = 120
img = image.img_to_array(image.load_img(sitting_data['data_addr'][i]))
label = scipy.io.loadmat(sitting_data['label_addr'][i])['M']
cut1,cut2,cut3 = random_put_into_256map(img,label)
test_img = cut3[:,:,0:3]
plt.figure()
plt.imshow(test_img/255)
plt.show()
input_img = np.expand_dims(test_img,axis=0)

# 2.Prediction

In [57]:
with g.as_default():
    pre = sess.run(pred_seg,feed_dict={model.img:input_img})
    outp = sess.run(out,feed_dict={model.img:input_img})

In [58]:
plt.figure()
plt.title('prediction')
plt.imshow(pre)
plt.show()

In [72]:
plt.figure()
plt.title('torso_activition')
plt.imshow(outp[:,:,2])
plt.show()

## 3.Guided Backpropagation

In [60]:
pre_mask = pre
map = np.zeros([64,64,15])
for i in range(15):
    for n in range(64):
        for m in range(64):
            if pre_mask[n][m] == i:
                map[n,m,i] = 1.0  

In [69]:
with g.as_default():
    np_mask = np.zeros([64,64,15],dtype='float32')
    #wrong_mask = np.zeros([64,64,15],dtype='float32')
    np_mask[:,:,2] = map[:,:,2]
    #wrong_mask[:,:,1] = 1
    mask = tf.constant(np_mask)
    #wrong_mask_t = tf.constant(wrong_mask)
    back_ground = out*mask
    #wrong_bg = out*wrong_mask_t
    gbp_back_ground = tf.gradients(back_ground,inpt)
    #wrong_back_ground = tf.gradients(wrong_bg,inpt)

In [70]:
with g.as_default():
    head = sess.run(gbp_back_ground,feed_dict={model.img:input_img})
    #wrong_head = sess.run(wrong_back_ground,feed_dict={model.img:input_img})
    supvision = sess.run(back_ground,feed_dict={model.img:input_img})

In [71]:
plt.figure()
plt.title('masked_torso_act')
plt.imshow(supvision[:,:,2])
plt.show()

In [73]:
gradient_abs = np.abs(head[0][0]).max(axis=-1)
plt.figure()
plt.title('head_abs')
plt.imshow(gradient_abs)
plt.show()

In [74]:
pos_gradient = (np.maximum(0, head[0][0]) / head[0][0].max())
#pos_gradient = pos_gradient*2/pos_gradient.max()
plt.figure()
plt.title('head_positive')
plt.imshow(pos_gradient)
plt.show()

In [75]:
neg_gradient = (np.maximum(0, -head[0][0]) / -head[0][0].min())
#neg_gradient = neg_gradient*2/neg_gradient.max()
plt.figure()
plt.title('head_negative')
plt.imshow(neg_gradient)
plt.show()

In [43]:
gradient_abs = np.abs(wrong_head[0][0]).max(axis=-1)
plt.figure()
plt.title('abs_wrong')
plt.imshow(gradient_abs)
plt.show()
pos_gradient = (np.maximum(0, wrong_head[0][0]) / wrong_head[0][0].max())
#pos_gradient = pos_gradient*2/pos_gradient.max()
plt.figure()
plt.title('positive_wrong')
plt.imshow(pos_gradient)
plt.show()
neg_gradient = (np.maximum(0, -wrong_head[0][0]) / -wrong_head[0][0].min())
#neg_gradient = neg_gradient*2/neg_gradient.max()
plt.figure()
plt.title('negative_wrong')
plt.imshow(neg_gradient)
plt.show()